In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

def output_results(out):
    pd.DataFrame({'deaths': out, 'Id': range(11)}).to_csv('mewmew_preds.csv', index=False)
    
def calculate_error(pred, real):
    return np.sqrt(np.mean((pred - real) ** 2))

training_df = pd.read_csv('phase1_training_data.csv')
training_df['date'] = training_df['date'].astype('datetime64')
training_df['country_id'].fillna('Unknown', inplace=True)

for col in training_df.columns[2:]:
    training_df[col+'_daily'] = training_df.groupby('country_id')[col].transform(lambda x: x.diff().fillna(0))
    
# ca_train_df = training_df[(training_df['country_id'] == 'CA') & (training_df['date'] >= '2020-03-15')]
ca_train_df = training_df[training_df['country_id'] == 'CA']

second_val_df = pd.read_csv('general_phase_1_answer.csv')
ca_val_df = second_val_df[second_val_df['country_id'] == 'CA']

In [2]:
new_training_df = pd.read_csv('general_phase_1_answer.csv')
new_training_df['country_id'].fillna('Unknown', inplace=True)
new_training_df = new_training_df[new_training_df['country_id'] == 'CA']

new_ca_train_df = pd.concat((ca_train_df, new_training_df[:-5]))

In [9]:
ca_val_df

,country_id,date,cases,deaths,cases_14_100k,cases_100k
640,CA,2020-10-06,168960,9504,59.60,451.63
641,CA,2020-10-07,171323,9530,63.00,457.95
642,CA,2020-10-08,173123,9541,64.23,462.76
643,CA,2020-10-09,175559,9557,67.10,469.27
644,CA,2020-10-10,178117,9585,70.69,476.11
645,CA,2020-10-11,178117,9585,66.80,476.11
646,CA,2020-10-12,178117,9585,60.99,476.11
647,CA,2020-10-13,182839,9627,69.17,488.73
648,CA,2020-10-14,186881,9654,75.17,499.53
649,CA,2020-10-15,189387,9664,77.12,506.23


In [8]:
from AutoReg_MultiVar import AutoRegressionModelMultiVar as armv

# errs = []

# i = 0
# for begining_ind in range(70, 180):
#     data = new_ca_train_df[begining_ind:]
#     deaths = data['deaths'].values
#     cases = data['cases'].values
#     for k_lag in range(1, 20):
#         for seq_case_count in range(20):
#             for case_lag in range(5, 30):
#                 lag_indices = list(np.arange(case_lag, min(case_lag+seq_case_count, 30)))
#                 model = armv(k=k_lag, bias=True)
#                 if len(lag_indices) == 0:
#                     model.fit(deaths, np.empty(0), np.empty(0))
#                 else:
#                     model.fit(deaths, [cases], [lag_indices])
#                 error = calculate_error(ca_val_df['deaths'][-5:].values, model.predict(len(data), len(data)+4))
#                 errs.append((begining_ind, k_lag, seq_case_count, case_lag, error))
#                 i += 1
#                 if i % 1000 == 0:
#                     print("i is:", i)

In [5]:
error_df = pd.DataFrame(errs, columns=['begining_index', 'k_lag', 'case_sequence_count', 'case_lag', 'error'])
error_df

,begining_index,k_lag,case_sequence_count,case_lag,error
0,70,1,0,5,15.095020
1,70,1,0,6,15.095020
2,70,1,0,7,15.095020
3,70,1,0,8,15.095020
4,70,1,0,9,15.095020
...,...,...,...,...,...
1044995,179,19,19,25,67.734946
1044996,179,19,19,26,61.544681
1044997,179,19,19,27,61.489326
1044998,179,19,19,28,54.288770


In [6]:
error_df.sort_values('error', inplace=True)
error_df

,begining_index,k_lag,case_sequence_count,case_lag,error
176069,88,11,2,24,1.445660
185569,89,11,2,24,1.523736
118579,82,10,3,9,1.577729
195069,90,11,2,24,1.631015
176093,88,11,3,23,1.653274
...,...,...,...,...,...
38053,74,1,2,8,287.003016
28553,73,1,2,8,290.190638
19053,72,1,2,8,293.550806
9553,71,1,2,8,296.609914


In [8]:
error_df.to_csv('final_errors.csv', index=False)

In [3]:
error_df = pd.read_csv('final_errors.csv')

In [4]:
err_count = []
for i in range(3, 20):
    err_count.append((i/2, sum(error_df['error'] < i / 2)))

pd.DataFrame(err_count, columns=['Validation Error', 'Number of models with less validation error'])

,Validation Error,Number of models with less validation error
0,1.5,1
1,2.0,46
2,2.5,283
3,3.0,1191
4,3.5,2569
5,4.0,5724
6,4.5,11758
7,5.0,21536
8,5.5,34748
9,6.0,51152


In [5]:
new_training_df = pd.read_csv('general_phase_1_answer.csv')
new_training_df['country_id'].fillna('Unknown', inplace=True)
new_training_df = new_training_df[new_training_df['country_id'] == 'CA']

new_ca_train_df = pd.concat((ca_train_df, new_training_df))

In [6]:
new_ca_train_df

,country_id,date,cases,deaths,cases_14_100k,cases_100k,cases_daily,deaths_daily,cases_14_100k_daily,cases_100k_daily
8960,CA,2019-12-31 00:00:00,0,0,0.00,0.00,0.0,0.0,0.0,0.0
8961,CA,2020-01-01 00:00:00,0,0,0.00,0.00,0.0,0.0,0.0,0.0
8962,CA,2020-01-02 00:00:00,0,0,0.00,0.00,0.0,0.0,0.0,0.0
8963,CA,2020-01-03 00:00:00,0,0,0.00,0.00,0.0,0.0,0.0,0.0
8964,CA,2020-01-04 00:00:00,0,0,0.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
655,CA,2020-10-21,203688,9794,81.70,544.46,NaN,NaN,NaN,NaN
656,CA,2020-10-22,206360,9829,82.33,551.60,NaN,NaN,NaN,NaN
657,CA,2020-10-23,209148,9862,82.95,559.05,NaN,NaN,NaN,NaN
658,CA,2020-10-24,211732,9888,89.85,565.96,NaN,NaN,NaN,NaN


In [59]:
rows_2 = []

preds = []
models = []
for ind, row in error_df[(error_df['error'] <= 5.5) & (error_df['error'] > 0)].iterrows():
    data = new_ca_train_df[int(row.begining_index):]
    deaths = data['deaths'].values
    cases = data['cases'].values
    model = armv(k=int(row.k_lag), bias=True)
    lag_indices = list(np.arange(row.case_lag, min(row.case_lag+row.case_sequence_count, 30)).astype('int'))
    model.fit(deaths, [cases], [lag_indices])
    preds.append([model.predict(len(data), len(data)+4)])
    if err < 2.25:
        rows_2.append(row)
    
# calculate_error(np.median(np.array(preds).reshape(-1, 5), axis=0), ca_val_df['deaths'][-5:].values)
print(np.median(np.array(preds).reshape(-1, 5), axis=0))

[ 9957.95878132  9997.78108314 10036.172621   10074.79234733
 10116.65063609]
19.4565481527193
21.246129847698754
0.8886314398610289


In [62]:
xx = pd.DataFrame(rows)
xx

,begining_index,k_lag,case_sequence_count,case_lag,error
1626,82.0,17.0,2.0,13.0,3.201511
1887,84.0,17.0,2.0,13.0,3.304589
1903,85.0,17.0,2.0,13.0,3.309554
2067,83.0,17.0,2.0,13.0,3.364563
2219,83.0,18.0,2.0,13.0,3.408313
2449,81.0,18.0,2.0,13.0,3.468904
2514,77.0,12.0,2.0,15.0,3.488981


In [65]:
x = pd.DataFrame(rows_2)
x

,begining_index,k_lag,case_sequence_count,case_lag,error
1626,82.0,17.0,2.0,13.0,3.201511
1887,84.0,17.0,2.0,13.0,3.304589
1903,85.0,17.0,2.0,13.0,3.309554
2067,83.0,17.0,2.0,13.0,3.364563
2219,83.0,18.0,2.0,13.0,3.408313
...,...,...,...,...,...
33334,122.0,10.0,7.0,5.0,5.454929
33732,122.0,19.0,9.0,7.0,5.469588
33817,92.0,18.0,3.0,12.0,5.473345
33899,84.0,18.0,7.0,19.0,5.476481


In [66]:
preds = []
models = []
for ind, row in x.iterrows():
    data = new_ca_train_df[int(row.begining_index):]
    deaths = data['deaths'].values
    cases = data['cases'].values
    model = armv(k=int(row.k_lag), bias=True)
    lag_indices = list(np.arange(row.case_lag, min(row.case_lag+row.case_sequence_count, 30)).astype('int'))
    model.fit(deaths, [cases], [lag_indices])
    preds.append([model.predict(len(data), len(data)+4)])
    
# calculate_error(np.median(np.array(preds).reshape(-1, 5), axis=0), ca_val_df['deaths'][-5:].values)
print(np.median(np.array(preds).reshape(-1, 5), axis=0))

[ 9947.36849722  9974.07564934 10003.06605134 10035.50114884
 10063.89704423]
1.2308139862247243
21.160959639232672
0.8886314398610289


In [67]:
print(np.median(np.array(preds).reshape(-1, 5), axis=0))

[ 9947.36849722  9974.07564934 10003.06605134 10035.50114884
 10063.89704423]


In [69]:
out = np.median(np.array(preds).reshape(-1, 5), axis=0)
pd.DataFrame({'deaths': out, 'Id': range(5)}).to_csv('final.csv', index=False)

In [71]:
np.diff(np.median(np.array(preds).reshape(-1, 5), axis=0))

array([26.70715212, 28.990402  , 32.43509749, 28.39589539])